# L5: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

عامل وظیفه را انجام می‌دهد:

عامل (مانند data_analyst_agent) وظیفه خود را انجام داده و نتیجه را به مدیر ارسال می‌کند.
مدیر نتیجه را بررسی می‌کند:

اگر نتیجه کافی باشد: مدیر آن را می‌پذیرد و به مرحله بعد می‌رود یا به عامل دیگری ارجاع می‌دهد.
اگر نتیجه ناکافی باشد: مدیر بازخوردی (Feedback) برای عامل ارسال می‌کند.
بازگرداندن وظیفه به عامل:

وظیفه به همراه بازخورد مشخص (مانند "داده‌ها ناقص هستند" یا "تحلیل دقیق‌تری نیاز است") به عامل مربوطه برمی‌گردد.
عامل وظیفه را بر اساس بازخورد تصحیح می‌کند و دوباره نتیجه را ارسال می‌کند.
تکرار فرآیند تا تأیید:

این چرخه ادامه پیدا می‌کند تا مدیر نتیجه را تأیید کند و وظیفه به مرحله بعد برود.


The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [ ]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}. Write in Persian"
    ),
    agent=data_analyst_agent,
)

In [ ]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance. Write in Persian"
    ),
    agent=trading_strategy_agent,
)


In [ ]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}. Write in Persian"
    ),
    agent=execution_agent,
)


In [ ]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}. Write in Persian"
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
# Example data for kicking off the process
financial_trading_inputs ={
    'stock_selection': 'EUR/USD',  # انتخاب جفت‌ارز اصلی و پرنوسان
    'initial_capital': '100000000 ریال',  # سرمایه اولیه (تقریباً 200 دلار آمریکا)
    'risk_tolerance': 'متوسط',  # تحمل ریسک در حد متوسط
    'trading_strategy_preference': 'Swing Trading'  # استراتژی معاملاتی روزانه
}


**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (EUR/USD). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to make sure that the team continuously monitors and analyzes market data for the selected stock (EUR/USD) in order to identify trends and predict market movements using statistical modeling and machine learning. I should delegate tasks to the appropriate team members and ask questions to gather all the necessary information.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for EUR/USD", "context": "Provide detailed analysis of market trends and movements using statistical modeling and machine learning techniques."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time data on the EUR/USD market to provide

باید به دقت استراتژی های معاملاتی مناسب برای EUR/USD را توسعه دهیم که با توانایی های تجارتی کاربر و تحمل ریسک تعیین شده توسط او همخوانی داشته باشد.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Develop potential trading strategies for EUR/USD based on market analysis and user-defined risk tolerance", "context": "Continuous analysis and prediction of market data for selected stocks (EUR/USD) is crucial. By using statistical modeling and machine learning, we can identify trends and predict market movements. With a detailed market analysis and the use of statistical modeling and machine learning techniques, we can identify important market patterns and opportunities or threats for EUR/USD and act effectively based on them."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for EUR/USD and analyze it using statistical modeling and machine learning to develop potential trading strategies. I should also consider the u

Final Answer:
Here are some reliable sources and platforms to gather real-time market data for EUR/USD:

1. EUR USD Technical Analysis - Investing.com
Link: https://www.investing.com/currencies/eur-usd-technical

2. Live Charts (Euro / US dollar) - Forex Live
Link: https://www.forexlive.com/LiveCharts

3. EUR/USD Overview: Price, News and How to Trade | IG International
Link: https://www.ig.com/en/forex/markets-forex/eur-usd

4. Forex APIs (Real-Time & Historical) - Twelve Data
Link: https://twelvedata.com/forex

5. EUR/USD Forecast & Live Chart Analysis | Trading Guide for EURUSD
Link: https://www.atfx.com/en/live-chart/eur-usd

6. EUR/USD (EURUSD FOREX) stock market data APIs
Link: https://eodhd.com/financial-summary/EURUSD.FOREX

7. Real-Time and Historical Foreign Currency Exchange Rates API
Link: https://www.xignite.com/product/forex

8. Live Forex Rates - FXStreet
Link: https://www.fxstreet.com/rates-charts/rates

9. Trade Forex Online with tastyfx: Low Spreads, 80+ Forex Pairs
L

Final Answer: 
After analyzing the market data for EUR/USD from Investing.com, it is evident that the EUR is currently in use by over 320 million Europeans across the continent, surpassing the USD in circulation value. The section on EUR/USD features live and historical data, charts, technical analysis, and a user area for discussions and sentiments. This information can help in determining the best execution methods for trading strategies related to EUR/USD.

> Finished chain.
 

After analyzing the market data for EUR/USD from Investing.com, it is evident that the EUR is currently in use by over 320 million Europeans across the continent, surpassing the USD in circulation value. The section on EUR/USD features live and historical data, charts, technical analysis, and a user area for discussions and sentiments. This information can help in determining the best execution methods for trading strategies related to EUR/USD.

Thought: 
I need to continue delegating tasks and gather more in

I need to thoroughly analyze the proposed trading strategies and execution plans for EUR/USD to identify potential risks and suggest mitigation strategies.

Action: Read website content
Action Input: {"website_url": "proposed_trading_strategies_URL"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'proposed_trading_strategies_URL': No scheme supplied. Perhaps you meant https://proposed_trading_strategies_URL?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to find information on proposed trading strategies for EUR/USD.
Action: Search the internet
Action Input: {"search_query": "proposed trading strategies for EUR/USD"} 


Search results: Title: How to Trade EUR/USD: Top 5 Tips to Boost Your Trading - Oanda
Link: https://www.oanda.com/us-en/trade-tap-blog/trading-knowledge/how-to-trade-eur-usd/
Snippet: Enhance your trading EUR/USD tr

Final Answer: 
Based on the information gathered from the search results, it is evident that the proposed trading strategies for EUR/USD involve various key factors such as tracking ECB and Fed monetary policy, understanding market risk appetite, analyzing economic data from the Eurozone and U.S., picking the right time to trade, and understanding key correlations. These strategies aim to enhance trading ability, predict market movements, and maximize trading opportunities. To mitigate risks associated with these strategies, traders should closely monitor central bank decisions, economic indicators, market sentiment, trading sessions, and correlations with other currency pairs. By following these tips and implementing effective risk management practices, traders can navigate the complexities of trading EUR/USD successfully.

> Finished chain.
 

Based on the information gathered from the search results, it is evident that the proposed trading strategies for EUR/USD involve various key 

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

گزارش تجزیه و تحلیل ریسک جامع که جزئیاتی راجع به ریسک های احتمالی و پیشنهادات کاهش ریسک برای EUR/USD ارائه می دهد:

بررسی عوامل مهمی که در استراتژی های معاملاتی برای EUR/USD دخیل هستند از جمله پیگیری سیاست های پولی ECB و Fed، درک تمایلات ریسک بازار، تحلیل داده های اقتصادی از منطقه یورو و ایالات متحده، انتخاب زمان مناسب برای معامله و درک همبستگی های کلیدی است. این استراتژی ها هدفشان افزایش توانایی معاملاتی، پیش بینی حرکات بازار و بیشینه سازی فرصت های معاملاتی است. برای کاهش ریسک های مرتبط با این استراتژی ها، معامله گران باید تصمیمات بانک مرکزی، شاخص های اقتصادی، حس بازار، جلسات معاملاتی و همبستگی های با سایر جفت ارزها را به دقت نظارت کنند. با پیروی از این نکات و اجرای شیوه های موثر مدیریت ریسک، معامله گران می توانند با موفقیت در معاملات EUR/USD حرکت کنند.